# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,johvi,59.3592,27.4211,5.51,45,38,3.69,EE,1715081444
1,1,tiksi,71.6872,128.8694,0.11,99,100,8.12,RU,1715081445
2,2,papatowai,-46.5619,169.4708,9.61,79,64,4.34,NZ,1715081445
3,3,urcos,-13.6861,-71.6228,8.94,85,55,0.79,PE,1715081445
4,4,nova sintra,14.8667,-24.7167,24.46,70,0,2.38,CV,1715081445


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size = "Humidity",
    geo = True,
    tiles = "OSM"
)
# Display the map
city_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[city_data_df["Cloudiness"] == 0]     #my ideal weather condition is zero cloudiness

# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna()

# Display sample data
ideal_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,nova sintra,14.8667,-24.7167,24.46,70,0,2.38,CV,1715081445
6,6,bethel,41.3712,-73.4140,16.12,72,0,2.06,US,1715081307
7,7,carnarvon,-24.8667,113.6333,22.56,60,0,8.42,AU,1715081445
13,13,nsanje,-16.9200,35.2620,34.30,33,0,2.60,MW,1715081385
28,28,le vauclin,14.5452,-60.8388,28.03,78,0,1.03,MQ,1715081448


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,johvi,EE,59.3592,27.4211,45,
1,tiksi,RU,71.6872,128.8694,99,
2,papatowai,NZ,-46.5619,169.4708,79,
3,urcos,PE,-13.6861,-71.6228,85,
4,nova sintra,CV,14.8667,-24.7167,70,
...,...,...,...,...,...,...
559,quelimane,MZ,-17.8786,36.8883,65,
560,mamuju,ID,-2.6748,118.8885,91,
561,lyantor,RU,61.6195,72.1555,100,
562,deltebre,ES,40.7499,0.7868,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"




params = {  "categories":categories,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()


    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
johvi - nearest hotel: Pääsuke
tiksi - nearest hotel: Арктика
papatowai - nearest hotel: No hotel found
urcos - nearest hotel: Hospedaje Rafael
nova sintra - nearest hotel: Residência Ka Dencho
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
bethel - nearest hotel: Hampton Inn Danbury
carnarvon - nearest hotel: No hotel found
darovskoy - nearest hotel: Центральная
mandalgovi - nearest hotel: Б
port-aux-francais - nearest hotel: Keravel
jamestown - nearest hotel: DoubleTree Jamestown
ribeira brava - nearest hotel: Cheerfulway Bravamar Hotel
nsanje - nearest hotel: No hotel found
vorgashor - nearest hotel: София
harstad - nearest hotel: Scandic Harstad
santa maria del oro - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
cidreira - nearest hotel: Hotel Castelo
plettenberg bay - nearest hotel: Milkwood Manor
badger - nearest hotel: Clarion Hotel & Suites
edinburgh of the seven seas - nearest hotel: No hotel found
lagoa - nearest hot

,City,Country,Lat,Lng,Humidity,Hotel Name
0,johvi,EE,59.3592,27.4211,45,Pääsuke
1,tiksi,RU,71.6872,128.8694,99,Арктика
2,papatowai,NZ,-46.5619,169.4708,79,No hotel found
3,urcos,PE,-13.6861,-71.6228,85,Hospedaje Rafael
4,nova sintra,CV,14.8667,-24.7167,70,Residência Ka Dencho
...,...,...,...,...,...,...
559,quelimane,MZ,-17.8786,36.8883,65,Mabassa
560,mamuju,ID,-2.6748,118.8885,91,Hotel d'Maleo
561,lyantor,RU,61.6195,72.1555,100,No hotel found
562,deltebre,ES,40.7499,0.7868,45,Delta Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)